In [ ]:
using Pipe
using DataStructures

In [ ]:
# task 1
input = read(open("test.txt"), String)
monkies = Vector{Queue{Int64}}()
monkeyInputs = @pipe split(input, "\n\n") |> map(x -> split(x, "\n"), _)
operations = Vector{String}()
tests = Vector{Tuple{Int64,Int64,Int64}}()
for mI in monkeyInputs
  startingItems = Queue{Int64}()
  for mm in eachmatch(r"(\d+)", mI[2])
    enqueue!(startingItems, parse(Int64, mm.match))
  end
  push!(monkies, startingItems)

  # operation = match(r"new = \d+|old +|* \d+|old", mI[3])
  operation = split(mI[3], ": ")[2]
  push!(operations, operation)

  divisibility = match(r"(\d+)", mI[4])
  ifTrue = match(r"(\d+)", mI[5])
  ifFalse = match(r"(\d+)", mI[6])
  push!(tests, (parse(Int64, divisibility.match), parse(Int64, ifTrue.match) + 1, parse(Int64, ifFalse.match) + 1))
end

new = 0
old = 0
rounds = 1000
inspections = fill(0, length(monkies))

for round in 1:rounds
  for i in 1:length(monkies)
    inspections[i] += length(monkies[i])
    (divisibility, ifTrue, ifFalse) = tests[i]
    while !isempty(monkies[i])
      old = dequeue!(monkies[i])
      Meta.parse(operations[i]) |> eval
      new = div(new, 3)

      if new % divisibility == 0
        enqueue!(monkies[ifTrue], new)
      else
        enqueue!(monkies[ifFalse], new)
      end
    end
  end
end

@pipe inspections |> sort |> _[length(_)] * _[length(_)-1]


In [ ]:
# task 2
# We can use that modulo prime arithmetic forms a field
# Maybe chinese remainder theorem?
input = read(open("input.txt"), String)
monkiesStarting = Vector{Queue{Int64}}()
monkeyInputs = @pipe split(input, "\n\n") |> map(x -> split(x, "\n"), _)
operations = Vector{Function}()
tests = Vector{Tuple{Int64,Int64,Int64}}()
for mI in monkeyInputs
  startingItems = Queue{Int64}()
  for mm in eachmatch(r"(\d+)", mI[2])
    enqueue!(startingItems, parse(Int64, mm.match))
  end
  push!(monkiesStarting, startingItems)

  operation = split(mI[3], ": ")[2]
  operationParts = split(operation)
  if operationParts[4] == "*"
    if operationParts[5] == "old"
      push!(operations, x -> x * x)
    else
      push!(operations, x -> x * parse(Int64, operationParts[5]))
    end
  else 
    if operationParts[5] == "old"
      push!(operations, x -> x + x)
    else
      push!(operations, x -> x + parse(Int64, operationParts[5]))
    end
  end

  divisibility = match(r"(\d+)", mI[4])
  ifTrue = match(r"(\d+)", mI[5])
  ifFalse = match(r"(\d+)", mI[6])
  push!(tests, (parse(Int64, divisibility.match), parse(Int64, ifTrue.match) + 1, parse(Int64, ifFalse.match) + 1))
end

# Each monkey's items have a vector of remainders
monkies = Vector{Queue{Vector{Int64}}}()
for i in 1:length(monkiesStarting)
  monkey = Queue{Vector{Int64}}()
  while !isempty(monkiesStarting[i])
    item = dequeue!(monkiesStarting[i])
    vecRemainders = Vector{Int64}()
    for (d, _, _) in tests
      push!(vecRemainders, item % d)
    end
    enqueue!(monkey, vecRemainders)
  end
  push!(monkies, monkey)
end


new = 0
rounds = 10000
inspections = fill(0, length(monkies))

for round in 1:rounds
  for i in 1:length(monkies)
    # display(monkies[i])
    inspections[i] += length(monkies[i])
    (_, ifTrue, ifFalse) = tests[i]
    while !isempty(monkies[i])
      new = dequeue!(monkies[i])
      # display(new)
      for j in 1:length(tests)
        new[j] = operations[i](new[j]) % tests[j][1]
        # new[j] = div(new[j], 3)
        # new[j] = div(operations[i](new[j]), 3) % tests[j][1]
      end

      if new[i] == 0
        enqueue!(monkies[ifTrue], new)
      else
        enqueue!(monkies[ifFalse], new)
      end
    end
  end
end

# display(monkies)
# display(inspections)
@pipe inspections |> sort |> _[length(_)] * _[length(_)-1]

In [ ]:
a = 2 > 1 ? "+" : "-"
eval("5 $a 2")
# eval(5 a 2)
@eval "5 + 2"
Meta.parse("5 $a 2") |> eval
